In [1]:
!pip install datasets evaluate transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 547.3 kB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=15ed1b1c25b031ddce5819df9b90edca265a2977458ed8f36b0bac64eabbd41e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd "/content/drive/MyDrive/week-4"

/content/drive/.shortcut-targets-by-id/1wCkFBZu8tASt7CdIfbiROqkWPsI1jC3g/week-4


In [7]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np


In [8]:
# Load the CoNLL-formatted file into memory
def load_conll_data(filepath):
    sentences = []
    current_tokens = []
    current_labels = []
    with open(filepath, encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line:  # blank line indicates end of a sentence
                if current_tokens:
                    sentences.append((current_tokens, current_labels))
                current_tokens, current_labels = [], []
            else:
                token, label = line.split(maxsplit=1)
                current_tokens.append(token)
                current_labels.append(label)
    print(f"Loaded {len(sentences)} labeled sentences")
    return sentences

sentences = load_conll_data("labeled_telegram_product_price_location_cleaned.conll")
# Split into train/validation sets (80/20 split)
train_sentences, val_sentences = train_test_split(sentences, test_size=0.2, random_state=42)
len(train_sentences), len(val_sentences)


Loaded 174645 labeled sentences


(139716, 34929)

In [9]:
from transformers import AutoTokenizer

# Choose a pre-trained model tokenizer (XLM-RoBERTa base for multilingual support)
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# Define label mappings
unique_labels = set(label for _, labels in sentences for label in labels) # Use all sentences to find unique labels
label_to_id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}
num_labels = len(label_to_id)
print(unique_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

{'O'}


In [10]:
def tokenize_and_align_labels(examples):
    # 1) Tokenize all sentences in this batch at once
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    # 2) Grab the NER labels from the batch
    all_labels = examples["labels"]  # or "ner_tags"
    new_labels = []

    # 3) For each sentence in the batch, build its aligned label list
    for i, labels in enumerate(all_labels):
        word_ids    = tokenized_inputs.word_ids(batch_index=i)
        prev_word   = None
        label_ids   = []

        for word_idx in word_ids:
            if word_idx is None:
                # special tokens
                label_ids.append(-100)
            elif word_idx != prev_word:
                # first subword of a new word
                label_ids.append(label_to_id[labels[word_idx]])
            else:
                # subsequent subword of the same word
                label_ids.append(-100)
            prev_word = word_idx

        new_labels.append(label_ids)

    # 4) Insert the aligned labels back into the tokenized batch
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

# Create Dataset objects from the loaded data
train_data = Dataset.from_dict({"tokens": [s[0] for s in train_sentences], "labels": [s[1] for s in train_sentences]})
val_data = Dataset.from_dict({"tokens": [s[0] for s in val_sentences], "labels": [s[1] for s in val_sentences]})

# Then map with batched=True:
tokenized_train = train_data.map(tokenize_and_align_labels, batched=True)
tokenized_val   = val_data.map(  tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/139716 [00:00<?, ? examples/s]

Map:   0%|          | 0/34929 [00:00<?, ? examples/s]

## Fine-Tuning the Transformer Model

In [11]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(unique_labels),
    id2label=label_to_id,
    label2id=label_to_id
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir="./ner-model",
    eval_strategy="epoch", # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
)

In [13]:
metric = evaluate.load("seqeval")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=2)
    refs  = p.label_ids
    true_p, true_r = [], []
    for pr, lb in zip(preds, refs):
        p_seq, l_seq = [], []
        for p_i, l_i in zip(pr, lb):
            if l_i != -100:
                p_seq.append(label_to_id[p_i])
                l_seq.append(label_to_id[l_i])
        true_p.append(p_seq); true_r.append(l_seq)
    res = metric.compute(predictions=true_p, references=true_r)
    return {
        "precision": res["overall_precision"],
        "recall":    res["overall_recall"],
        "f1":        res["overall_f1"],
        "accuracy":  res["overall_accuracy"]
    }

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator, # Add data collator
    compute_metrics=compute_metrics
)

trainer.train()
print(trainer.evaluate())
trainer.save_model("amharic-ner-model")

/tmp/ipython-input-15-1862554640.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eyosimar524 (eyosimar524-kifiya-financial-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })